<a href="https://colab.research.google.com/github/WilliamYkZhang/COMP551_A2/blob/master/model_selection_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Preprocessing
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
import pandas as pd

# Transformers 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from sklearn.decomposition import TruncatedSVD

# Models 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# Module to write final params 
import csv
import datetime
import pickle 

In [2]:
# Download stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mlej\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
def grid_search_cv(model, X, y, params, folds):
    # Pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(smooth_idf=True, norm='l2', lowercase=True, max_features=30000, use_idf=True, encoding = "utf-8",  decode_error = 'ignore', strip_accents='unicode', stop_words=stopwords.words('english'), analyzer = "word")),
        ('clf', model)],
         verbose=True)

    # Use GridSearch cross validation to find the best feature extraction and hyperparameters
    gs_CV = GridSearchCV(pipeline, param_grid=params, cv=folds)
    gs_CV.fit(X, y)
    print("Performing grid search...")
    print("Pipeline: ", [name for name, _ in pipeline.steps])
    print("Best parameter (CV score={0:.3f}):".format(gs_CV.best_score_))
    print("Best parameters set: {} \nBest estimator parameters {}.".format(gs_CV.best_params_, gs_CV.best_estimator_.get_params()))

    return (gs_CV.best_score_,gs_CV.best_params_, gs_CV.best_estimator_.get_params())

In [5]:
# Parameters
parameters_tfidf = {
'clf__booster': ('gbtree', 'gblinear'),
'clf__objective':('multi:softmax', 'binary:logistic'),   
# 'clf__eval_metric':('rmse', 'mae', 'logloss', 'logloss', 'error', 'merror', 'mlogloss', 'auc'),
# 'clf__seed': (0,1,2,3),
'clf__eta': (0.3,0.5), # learning rate
'clf__min_child_weight':(1,3,5,10), # Used to control over-fitting. Higher values prevent a model from learning relations which might be highly specific to the particular sample selected for a tree. Too high values can lead to under-fitting hence, it should be tuned using CV.
'clf__max_depth': (3,6,9),
}  

# Instantiate model
clf = xgb.XGBClassifier()

AttributeError: module 'xgboost' has no attribute 'XGBClassifier'

In [ ]:
# Read DataFrame
stemmed_df = pd.read_csv("https://raw.githubusercontent.com/WilliamYkZhang/COMP551_A2/master/preprocessed_reddit_train_SnowballStemmer.csv?token=AKKZG4GENVP2WXEXXHZVHSS5WRRRY")

# Separate X and Y 
X_stem = stemmed_df["cleaned"]
y_stem = stemmed_df["label"]

In [ ]:
best_scores, best_params, best_estimator_params = grid_search_cv(model=clf, X=X_stem, y=y_stem,params=parameters_tfidf,folds=5)